In [1]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [4]:
# Load data from the Excel file
excel_file_path = "../../DATAFORMODELtrain200824.xlsx"
data = pd.read_excel(excel_file_path)
# Display the first few rows to understand the structure
print(data.head())